# Deep Reinforcement Learning
dt.: *Bestärkendes Lernen*

Für wen?

Schon einfache Computerspiele sind als Umgebung zu komplex um mit einfacher Klassifizierung zu einer brauchbaren Lösung zu kommen. Ursachen:

* *supervised*: Jede Trainingsinstanz hat ein Label.
  * Sogar in der einfachen Umgebung eines Computerspieles bräuchte man sehr viele Instanzen -> Screenshots für jede Situation/Spielzustand. 
  * Spielzüge, die sich erst später im Spiel auswirken bleiben unberücksichtigt.
* *Reinforcement Learning*: zwischen *supervised* und *unsupervised* *learning*. Manche Zustände bringen *rewards*. Diese Rewards können als (spärliche und zeitversetze) Labels betrachtet werden. Über *Reinforcement Learning* werden Wege dorthin durch Bewertung von Zuständen, die für sich genommen noch keinen Erfolg bringen, gelernt.

### Schwierigkeiten:
* **credit assignment problem**: Welche der Züge waren für das Bekommen der Belohnung verantwortlich?
* **explore-exploit dilemma**: soll eine einmal gefundene, funktionierende Strategie beibehalten werden (*exploitation*), oder ist es zielführend eine nach einer besseren zu suchen (*exploration*)?

# Markov Decision Process (MDP)

Eine übliche Art den Sachverhalt zu beschreiben ist die Betrachtung als *Markov Decision Process*.


* Aktionen (***actions***) führen zu einem neuen Zustand (***state***) und können die Umgebung verändern.
* *actions* führen manchmal zu Belohnungen durch das *environment* oder einen Interpreter des Zustands
 

<img src="images/actor_state_environment.png" alt="[Die Reinforcement Learning Komponenten: Actor State Environment" style="width: 300px;"/>
Die Reinforcement Learning Komponenten: Actor, State, Environment<br>Source: selfmade

* Die Regeln zur Wahl einer Aktion werden unter ***policy*** zusammengefasst.
* Die Umgebung ist nicht zwingend deterministisch, dh. ausgehend vom gleichen Zustand führt eine Aktion nicht immer zum gleichen Folgezustand (Zufallselemente -> stochastisches Verhalten)
* Eine *Episode* ist die Abfolge von *states*, *actions*, *rewards* bis zum Spielende (finaler Zustand $s_n$): $s_0,a_0,r_1, s_1,a_1,r_2, ...s_{n-1},a_{n-1},r_n,s_n$

<img src="images/markov_decision_process.png" alt="Markov Decision Process Graph" style="width: 300px;"/>
Markov Decision Process Graph<br>Source: Wikipedia

* Wichtige Eigenschaft von *MDP*: Der *MDP* basiert auf der Annahme, daß der nächste Zustand $s_{i+1}$ (bzw. die Wahrscheinlichkeit dafür) ausschließlich vom derzeitigen Zustand $s$ und der gewählten Aktion $a$ abhängt, nicht aber von den Zuständen davor.

# Zukünftige Belohnungen diskontieren

Um langfristig gut abzuschneiden müssen zukünftige potentielle Belohnungen miteinbezogen werden.

Der *reward* $R$ einer gesamten Episode lässt sich einfach berechnen: $R = r_1 + r_2 +  .. + r_n$, bzw. für den verbleibenden Spielverlauf: $R_t = r_t + r_{t+1} +  .. + r_n$

Wie der zukünftige Verlauf ist, lässt sich aber nicht zuverlässig vorhersagen (stochastische Umgebung). Daher ist es üblich die kommende Belohnung nur mit einer bestimmten Wahrscheinlichkeit $\gamma$ zu berücksichtigen und die darauf folgende Belohnung wieder mit der gleichen Wahrscheinlichkeit, was zu $\gamma^2$ führt, usw.:

### $R_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} +  .. + \gamma^{n-t}r_n$

Dh. die kommenden Belohnungen werden diskontiert. Kompakter dargestellt:

### $\displaystyle R_t = \sum^{N-1}_{t=0}\gamma^t\cdot r_{t+1}$

$\gamma$ ist ein Wert zwischen $0$ und $1$. Dh. je weiter in der Zukunft eine Belohnung ist, desto geringer ist ihr Einfluss. Je näher $\gamma$ bei 1 ist, desto stärker werden zukünftige Belohnungen berücksichtigt. In einer deterministischen Umgebung kann zB. $\gamma=1$ gesetzt werden. Wenn $\gamma=0$ wird nur die unmittelbar nächste Belohnung in Betracht gezogen.

### $R_t = r_t + \gamma(r_{t+1} + \gamma(r_{t+2} + ...)) = r_t + \gamma R_{t+1}$

Nun gilt es die Aktionen so zu wählen, dass $R_t$ maximiert wird, und hier kommt Q-Learning ins Spiel!

# Q-Learning

**$Q(s, a)$** ist der ***discounted future reward***, wenn die Aktion $a$ im Zustand $s$ gesetzt wird und alle **Folgeaktionen optimal gewählt** sind. Woher wissen wir, dass die Aktionen optimal gewählt wurden, oder welche Aktionen optimal sind? Zunächst wissen wir es gar nicht. Aber wir können uns hinarbeiten.

### $Q(s_t, a_t) = max_\pi R_{t+1}$

$Q(s, a)$ ergibt, aus jetziger Sicht (dh. Zustand $s$), das bestmögliche Ergebnis am Ende des Spiels, wenn im Zustand $s$ die Aktion $a$ gewählt wird. Damit fällt die Wahl der optimalen Aktion leicht: die Aktion, die den höchsten Q-Wert ergibt:

### $\pi(s) = argmax_a Q(s, a)$

Bleibt noch zu klären, wie man zu Q kommt. Wie beim *discounted future reward* lässt sich formulieren:

### Bellman Equation: $Q(s, a) = r + \gamma max_{a'} Q(s', a')$

Das erscheint logisch: Die maximale zukünftige Belohnung für diesen Zustand und diese Aktion ist die unmittelbare Belohnung plus die maximale zukünftige Belohnung des nächsten Zustandes $s'$. Die maximalie zukünftige Belohnung wird ganz einfach bestimmt, indem für den nächsten Zustand $s'$ die Aktion $a'$ gewählt wird, für die es die größte Belohnung gibt.

* Iterative Annäherung an die Q-Funktion mit Hilfe der Bellman Gleichung
* Im einfachsten Fall werden die Q-Werte $Q[s', a']$ in einer Tabelle (*states* x *actions*) gehalten.
* Algorithmus: solange, bis die Episode aus ist:
  1. Aktion a setzen
  2. Es ergeben sich Belohnung $r$ und Zustand $s'$
  3. Update der Tabelle mit $Q[s, a] = Q[s, a] + \alpha(r+\gamma\, max_{a'} Q[s', a'] - Q[s, a])$
  4. $s = s'$
* $\alpha$ ist die Lernrate die bestimmt, wie sehr der bisherige Wert $Q[s,a]$ im neuen/aktualisierten $Q[s, a]$ Wert enthalten bleibt. Ist $\alpha=1$ so neutralisieren sich die beiden $Q[a,s]$ und der bisherige Wert $Q[s,a]$ hat gar keinen Einfluss auf den aktuellen und es bleibt die oben genannte *Bellmann Equation*.
* Zu Beginn des Trainings wird $max_{a'} Q[s', a']$ nichts Brauchbares liefern, da in der Q-Tabelle (sozusagen das Gedächtnis des *agents*) nach der Initialisierung noch nichts stehen kann, das mit der Umgebung in Zusammenhang steht. Die Werte werden jedoch mit jeder Iteration passender und werden, ausreichend Wiederholungen vorausgesetzt, gegen den wahren Q-Wert konvergieren.
* Mit jeder Episode wird dazu gelernt


# Umsetzungsmöglichkeiten: von der einfachen Tabelle zum Deep Q Network

Einfaches Spiel - einfache Umgebung zB. Spielfeld mit 4x4 Feldern, Bewegungsmöglichkeit in vier Richtungen -> 16 Zustände, 4 Aktionen -> 16x4 Tabelle $Q$.

Verallgemeinerung: Pixel + Farbabstufungen als Felder. Um Geschwindigkeiten zu erfassen 4 Screenshots. Jede Pixel-Farb-Layer-Kombination stellt einen Zustand dar. Bsp. 64x64 Bild mit 256 Graustufen und 4 Screen shots: $Farben^{res_x \cdot res_y \cdot layer} = 256^{64x64x4}$.

Viele der auftretenden Zustände kommen nur selten vor, dh. selbst nur die besuchten Zustände in einer Tabelle zu berücksichtigen (sparse Table) ist wenig zielführend. Viele Zustände werden nie besucht.

<hr>
Kombinatorikhilfe:<br>
Zahlensystem mit 256 Zuständen pro Stelle. So wie im Dezimalsystem 2 Stellen 100, also $10^2$ unterschiedliche Zustände ermöglichen, hat man im 256er System bei 2 Stellen $256^2$ Zustandsmöglichkeiten. Bei 64x64 Pixel, also 64x64 Stellen kommt man auf das gigantische Ergebnis von $256^{64 x 64 x 4}$ Zuständen, also um ein Vielfaches mehr als das Universum Atome hat.
<hr>

Weitere Verallgemeinerung - Sinnesorgane: Sehfeld, Gehör, Tastsinn, Geruchssinn

Es hat hier auch ein Wechsel von der tatsächlichen Umgebung zur Wahrnehmung der Umgebung stattgefunden.

Die $Q$-Funktion kann durch ein neuronales Netzwerk abgebildet werden, mit dem Zustand (4 Screens) und der Aktion als Input und dem Q-Wert als Ergebnis. Um in einem Schritt für alle Aktionen Q-Werte zu bekommen, kann alternativ der Zustand als Input genommen werden mit Q-Werten für jede Aktion als Ergebnis (verifizieren: DeepMind Ansatz).

![](images/deep_q_learning.png "")
Deep Q Learning<br>
Source: selfmade

Jeder Q-Wert ist ein reeller Wert. Die Fehlerfunktion, mit der die Differenz zwischen Zielwert ($r + \gamma\, max_{a'} Q(s', a')$) und Voraussage ($Q(s, a)$) minimiert werden soll, kann zB. so aussehen:

###  $L = \frac{1}{2}[r + \gamma\, max_{a'} Q(s', a') - Q(s, a)]^2$

Abgeänderter Algorithmus für den Schritt von $s$ zu $s'$:
* *feedforward pass*: füttern des NN mit dem derzeitigen Zustand um die Q-Werte für alle Aktionen zu bekommen.
* *feedforward pass*: füttern des NN mit $s'$ und von den zurückgelieferten Q-Werten den größten berechnen:  $argmax_{a'} Q(s', a')$
* damit den Zielwert $r + \gamma\, max_{a'} Q(s', a')$ für die Aktion $a$ berechnen. Für die nicht gewählten Aktionen wird der Zielwert auf den Wert, den das NN geliefert hat gesetzt, damit die Fehlerfunktion als Fehler $0$ berechnet.
* Gewichte aktualisieren (backpropagation)

## Netzwerk Topologie
Was wäre eine passende Netzwerk-Topologie? Im DeepMind Atari Paper wurde folgender Aufbau gewählt:

|Layer |Input|Filter size | Stride | Num filters | Activation | Output|
|------|-----|------------|--------| ------------|------------|-------|
|conv1	|84x84x4	|8×8  |	4 |	32	|ReLU	|20x20x32 |
|conv2	|20x20x32	|4×4  |	2 |	64	|ReLU	|9x9x64   |
|conv3	|9x9x64	    |3×3  |	1 |	64	|ReLU	|7x7x64   |
|fc4	|7x7x64		|	  |   |512  |ReLU	|512      |
|fc5	|512		|	  |   |18	|Linear	|18       |

Nachdem für diesen Anwendungsfall die Positionierung der erkannten Features wichtig ist, werden keine Pooling Layer verwendet.

Was hindert uns noch loszulegen? Nichts, allerdings wäre es wohl ein frustrierendes Unterfangen. Um die Q-Werte zum konvergieren zu bringen braucht es einige Tricks und eine Menge Rechenpower.

## Experience Replay

Aufeinanderfolgende Situationen  (*transitions*) sind oft recht ähnlich und führen mitunter zu lokalen Minima.

Um das zu verhindern ist die wichtigste Technik ***experience replay***: Während des Spiels werden die Erfahrungen ($s, a, r, s'$) gespeichert (*replay memory*). Für das Trainieren des Netzwerkes werden anschließlich zufällig Werte aus dem *replay memory* genommen.

Die Erfahrungen können auch von menschlichen Spielern gesammelt werden.
Geht in Richtung *supervised learning*.

## Exploration-Exploitation

Die erstbeste erfolgreiche Strategie wird genommen und beibehalten weil sie den höchsten Q-Wert hat - *greedy*.

Abhilfe:
Anstatt immer die Aktion mit dem höchsten Q-Wert zu wählen, wird mit einer geringen Wahrscheinlichkeit $\epsilon$ die Aktion zufällig gewählt: ***$\epsilon$-greedy exploration***

DeepMind hat $\epsilon$ von $1$ beginnend während des Spielens bis $0.1$ abnehmen lassen. Zu Beginn werden so Werte gesammelt, der Zustands-Raum ausgelotet, gegen Ende hin geht es um das Konvergieren lassen der gefundenen Pfade.

## Deep Q-Learning Algorithmus

bis zu Spielende wiederholen:
1. im Zustand $s$ eine Aktion $a$ wählen
  * Mit Wahrscheinlichkeit $\epsilon$ eine zufällige Aktion wählen
  * andernfalls $a = argmax_{a'} Q(s, a')$
2. Aktion $a$ durchführen
3. *reward* $r$ und neuer Zustand $s'$ aus dem Environment (oder einem Interpreter) erfassen
4. Erfahrung ($s, a, r, s'$) im *replay memory* $D$ speichern
5. zufälligen Zustandwechsel aus dem *replay memory* wählen